In [14]:

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import datetime


In [15]:
#arrestdata json file
dfa = pd.read_json('./data/arrestData.json')

#player info by year
dfr = pd.read_csv('./data/regular_season_rosters.csv')



In [16]:
def crimeCategoryCount(i, category):
    if category:
        dfa.set_value(i, category, 1)
        
[crimeCategoryCount(x, y) for x, y in dfa['Crime_category'].iteritems()]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [17]:

dfr.columns = ['No','name','Age','Pos','G','GS','Wt','Ht','College','BirthDate','Yrs','AV','Drafted (tm/rnd/yr)','team','year']

#Seperate information about draft pick into seperate columns
dfr[['Draft_Team', 'Draft_Round', 'Draft_Pick','Draft_Year']] = dfr['Drafted (tm/rnd/yr)'].str.split('/', expand=True)
dfr = dfr.drop('Drafted (tm/rnd/yr)', axis=1)
#data cleanup
dfr['team'] = dfr['team'].str.upper()
dfr['Pos'] = dfr['Pos'].str.upper()
dfr['name'] = dfr['name'].str.replace(r'[/*+]','')
dfr['Draft_Pick'] = dfr["Draft_Pick"].str.replace(r'[^0-9]','')
dfr['Draft_Year'] =  dfr['Draft_Year'].str.replace(r'[^0-9]','')
dfr['Draft_Round'] = dfr["Draft_Round"].str.replace(r'[^0-9]','')
dfr['Yrs'] = dfr['Yrs'].replace("Rook", 0)



#New(from arrest data) to Old(From pro football reference)
teamMap = { 'ARI':'CRD', 
           'NE':'NWE', 
           'BAL':'RAV', 
           'LAC':'SDG', 
           'NO':'NOR', 
           'SF':'SFO', 
           'GB':'GNB',
           'OAK':'RAI',
           'LA':'RAM',
           'HOU':'HTX', 
           'IND':'CLT', 
           'KC':'KAN',
           'TEN':'OTI',
           'JAC': 'JAX',
           'TB':'TAM'
          }

def teamNameReplace(new, old):
    dfr['team'] = dfr['team'].replace(old,new)
    

{teamNameReplace(new, old) for new, old in teamMap.items()}





{None}

In [18]:
grouped = dfa.groupby(['Name', 'Year', 'Team'])#['Category'].count()
test = grouped.agg({'Category' : 'count', 'DUI' : 'count','Other' : 'count','Drugs':'count', 'Domestic Violence':'count','Resisting':'count','Assault / Battery':'count','Gun':'count','Alcohol':'count','License / Traffic':'count','Disorderly conduct':'count','Sex':'count','Theft / Burglary':'count','Murder / Manslaughter':'count','Animal Abuse':'count'})
print(type(test))
arrestCountByYear = pd.DataFrame(test).reset_index()

####Example Lookup######
#arrestCountByYear[(arrestCountByYear.Name == "David Boston")]
#arrestCountByYear.sample(50)


dfm = pd.merge(dfr, arrestCountByYear, how="outer", left_on=['name', 'year', 'team'], right_on=['Name', 'Year', 'Team'])

#Select players who were arrested and not matched with a regular season roster but still active between 2000-2013.
#aka don't have profile info
dfMissing = dfm[(dfm.Name.notnull()) & (dfm.name.isnull())]

#Shows the remaining players that have profile info
dfFull = dfm[(dfm.name.notnull())]

dfm.to_csv('groupermerge.csv')

#####testing accuracy######## 
#print(len(dfFull) + len(dfMissing))
#print(len(dfm))



<class 'pandas.core.frame.DataFrame'>


In [19]:
len(arrestCountByYear)

761

## Replace Names
Most of the players below can be matched on name to data that appears above.
Players that could not be matched were the result of:
- Stemming issues (ie. Joe to Joseph)
- They did not play on a regular season roster during the period of 2000-2013

In [20]:
import datetime
dfm['name'] = dfm['name'].str.replace("'","")

def fillBlankProfiles(i,playerRow):
    #print(playerRow[['Name','Year','Team']])
    #print("Found matches in merged dataframe")
 
    lookupDf = dfm[dfm.name.str.upper() == playerRow['Name'].upper()].reset_index()
    
    ##For those whose names appear in the origal dataframe pick the closest year and fill with that data.
    if not(lookupDf.empty):
        x = (abs(lookupDf['year'] - playerRow['Year'])).idxmin()
        dfMissing.set_value(i, 'name', lookupDf.iloc[x]['name'])
        ##age will be calculated
        dfMissing.set_value(i, 'Pos', lookupDf.iloc[x]['Pos'])
        dfMissing.set_value(i, 'G', lookupDf.iloc[x]['G'])
        dfMissing.set_value(i, 'GS', lookupDf.iloc[x]['GS'])
        dfMissing.set_value(i, 'Wt', lookupDf.iloc[x]['Wt'])
        dfMissing.set_value(i, 'Ht', lookupDf.iloc[x]['Ht'])
        dfMissing.set_value(i, 'College', lookupDf.iloc[x]['College'])
        dfMissing.set_value(i, 'BirthDate', lookupDf.iloc[x]['BirthDate'])
        dfMissing.set_value(i, 'Yrs', lookupDf.iloc[x]['Yrs'])
        dfMissing.set_value(i, 'AV', lookupDf.iloc[x]['AV'])
        dfMissing.set_value(i, 'No', lookupDf.iloc[x]['No'])
        dfMissing.set_value(i, 'team', playerRow['Team'])
        dfMissing.set_value(i, 'year', playerRow['Year'])
        dfMissing.set_value(i, 'Draft_Team', lookupDf.iloc[x]['Draft_Team'])
        dfMissing.set_value(i, 'Draft_Round', lookupDf.iloc[x]['Draft_Round'])
        dfMissing.set_value(i, 'Draft_Pick', lookupDf.iloc[x]['Draft_Pick'])
        dfMissing.set_value(i, 'Draft_Year', lookupDf.iloc[x]['Draft_Year'])
        #print(str(playerRow['BirthDate'])+'-'+str(playerRow['name'])+'-'+str(playerRow['year']))
        whyear  = datetime.datetime.strptime(str(lookupDf.iloc[x]['BirthDate']), "%m/%d/%y").year
        if whyear > 2000: #some issues with datetime saying 
            whyear = whyear-100
        dfMissing.set_value(i, 'Age', lookupDf.iloc[x]['year']-whyear)
        
            
            
            

        
    
[fillBlankProfiles(i, playerRow) for i,playerRow in dfMissing.iterrows()]

dfMissing.to_csv('FillerOut.csv')

In [21]:
dfx = dfMissing[dfMissing.name.notnull()]
dfFinal = dfFull.append(dfx)
##get rid of matching columns for Team, Name, Year

dfFinal = dfFinal.drop(['Team', 'Name', 'Year'], axis=1)



list(dfFinal)


['No',
 'name',
 'Age',
 'Pos',
 'G',
 'GS',
 'Wt',
 'Ht',
 'College',
 'BirthDate',
 'Yrs',
 'AV',
 'team',
 'year',
 'Draft_Team',
 'Draft_Round',
 'Draft_Pick',
 'Draft_Year',
 'Category',
 'DUI',
 'Other',
 'Drugs',
 'Domestic Violence',
 'Resisting',
 'Assault / Battery',
 'Gun',
 'Alcohol',
 'License / Traffic',
 'Disorderly conduct',
 'Sex',
 'Theft / Burglary',
 'Murder / Manslaughter',
 'Animal Abuse']

In [22]:
dfFinal['BirthDate'] = dfFinal['BirthDate'].apply(lambda x: datetime.datetime.strptime(x,"%m/%d/%y").toordinal() if type(x)==str else x)

In [23]:
dfFinal['Pos'] = dfFinal.Pos.astype(str)
dfFinal['team'] = dfFinal.team.astype(str)
dfFinal['Draft_Team'] = dfFinal.Draft_Team.astype(str)
dfFinal['College'] = dfFinal.College.astype(str)

In [24]:
dfFinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33282 entries, 0 to 33363
Data columns (total 33 columns):
No                       32879 non-null float64
name                     33282 non-null object
Age                      33276 non-null float64
Pos                      33282 non-null object
G                        33282 non-null float64
GS                       33250 non-null float64
Wt                       33275 non-null float64
Ht                       815 non-null float64
College                  33282 non-null object
BirthDate                33276 non-null float64
Yrs                      33282 non-null object
AV                       33281 non-null float64
team                     33282 non-null object
year                     33282 non-null float64
Draft_Team               33282 non-null object
Draft_Round              23776 non-null object
Draft_Pick               23776 non-null object
Draft_Year               23753 non-null object
Category                 679 non-null 

In [25]:
dfFinal.sample()

,No,name,Age,Pos,G,GS,Wt,Ht,College,BirthDate,...,Resisting,Assault / Battery,Gun,Alcohol,License / Traffic,Disorderly conduct,Sex,Theft / Burglary,Murder / Manslaughter,Animal Abuse
13636,42.0,Sam Brandon,24.0,FS,16.0,10.0,200.0,NaN,UNLV,722635.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
dfFinal.to_csv('./data/final.csv', index=False)